In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm, notebook
import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/crypto-news-mar-2023/articles-crypto.csv


#### Code for fetching data from news api.

In [2]:
# !python -m pip install newsapi-python

# from newsapi import NewsApiClient

# API_KEY = '856737b219bf4835acc6352ea966c67b'
# newsapi = NewsApiClient(api_key=API_KEY)

# from datetime import datetime, timedelta

# def get_past_time_range(days=30):
#     past_days = {}
#     for day in range(1, days):
#         from_day = str(datetime.now().date() - timedelta(days=day))
#         to_day = str(datetime.now().date() - timedelta(days=day - 1))
#         past_days.update({from_day:to_day})
#     return past_days

# get_past_time_range(30)

# def get_articles(query, days=30):
#     past_days_filter = get_past_time_range(days)
#     all_articles = []
#     for fromDate,toDate in notebook.tqdm(past_days_filter.items()):
#         for page in range(1, 3):
#             try:
#                 page_articles = newsapi.get_everything(q=query, language='en', from_param=fromDate, to=toDate,sort_by='relevancy', page=page)['articles']
#                 if len(page_articles) == 0: break
#                 all_articles.extend(page_articles)
#             except:
#                 break
            
#     return all_articles

# articles = get_articles('cryptocurrency')

# len(articles)

# articles[0]

# articles_df = pd.DataFrame(articles)
# articles_df.head()

# articles_df.to_csv('articles-crypto.csv')

In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c3bffd7f5aeaeaed4268a48724abd4bc3fb99ce10b0d20bf1e9f1a30e849eecf
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers


In [4]:
from sentence_transformers import SentenceTransformer

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
articles_df = pd.read_csv('/kaggle/input/crypto-news-mar-2023/articles-crypto.csv')
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16T05:00:03Z,On 28 October 2022 Elon Musk tweeted the bird ...
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16T11:34:00Z,Silicon Valley had $74.9 billion in venture-ca...
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16T15:00:17Z,"If you own an Android smartphone, then the pla..."
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16T13:30:17Z,Crypto assets have existed for over a decade b...
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16T10:12:06Z,"OKB, the utility token of popular cryptocurren..."


In [7]:
titles = articles_df['title'].values.tolist()
titles[:5]

['Is the bird really freed? A look back at six months of Musk’s Twitter reign',
 'Remote Working Increases VC Investments in Other Areas Besides Silicon Valley',
 'Despite the best efforts of Google Play Protect, the Play Store is not as safe as it may appear',
 '5 Myths About Crypto Regulations Debunked',
 "OKB Unfazed By BTC's Retreat, Gains 50% In Weekly Charts"]

In [8]:
embeddings = model.encode(titles)
embeddings[:5]

Batches:   0%|          | 0/177 [00:00<?, ?it/s]

array([[ 0.02549198,  0.08546034,  0.08514886, ..., -0.08972151,
         0.02734277, -0.00249079],
       [ 0.10810093, -0.08295441,  0.06143335, ..., -0.09770083,
        -0.03455804, -0.02146049],
       [-0.08645321,  0.02147196,  0.01853899, ..., -0.07560316,
         0.12599476,  0.13990094],
       [-0.0264598 ,  0.01820142, -0.04395117, ..., -0.03163949,
         0.05570024,  0.02574591],
       [ 0.00991424, -0.00632995,  0.07007281, ..., -0.12255175,
        -0.06314813, -0.00612311]], dtype=float32)

In [9]:
len(embeddings), len(embeddings[0])

(5653, 384)

Reference link - https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e

In [10]:
!pip install umap-learn
!pip install hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=3427920 sha256=93960db0e0a4c20007db1bd2331

In [11]:
import umap
import hdbscan

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components, 
                      min_cluster_size,
                      random_state = None):
    """
    Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP
    """
    umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors, 
                                n_components=n_components, 
                                metric='cosine', 
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean', 
                               cluster_selection_method='eom').fit(umap_embeddings)

    return clusters

In [13]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given cluster supplied from running hdbscan
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [14]:
def random_search(embeddings, space, num_evals):
    """
    Randomly search hyperparameter space and limited number of times 
    and return a summary of the results
    """
    
    results = []
    
    for i in notebook.trange(num_evals):
        n_neighbors = random.choice(space['n_neighbors'])
        n_components = random.choice(space['n_components'])
        min_cluster_size = random.choice(space['min_cluster_size'])
        
        clusters = generate_clusters(embeddings, 
                                     n_neighbors = n_neighbors, 
                                     n_components = n_components, 
                                     min_cluster_size = min_cluster_size, 
                                     random_state = 97)
    
        label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
                
        results.append([i, n_neighbors, n_components, min_cluster_size, 
                        label_count, cost])
    
    result_df = pd.DataFrame(results, columns=['run_id', 'n_neighbors', 'n_components', 
                                               'min_cluster_size', 'label_count', 'cost'])
    
    return result_df.sort_values(by='cost')

In [15]:
space = {
    "n_neighbors": range(12, 16),
    "n_components": range(3, 7),
    "min_cluster_size": range(2, 16),
    "random_state": 97
}
results = random_search(embeddings, space, 100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [16]:
results

,run_id,n_neighbors,n_components,min_cluster_size,label_count,cost
56,56,13,3,4,476,0.102247
83,83,12,6,5,401,0.102777
5,5,13,5,3,658,0.105961
89,89,12,3,3,622,0.110915
0,0,12,5,3,644,0.112153
...,...,...,...,...,...,...
97,97,15,6,15,113,0.238811
12,12,12,6,15,129,0.239873
53,53,15,4,15,117,0.251371
44,44,15,3,15,122,0.275783


In [18]:
clusters = generate_clusters(embeddings, 15, 6, 15, 97)
len(clusters.labels_), len(embeddings)

(5653, 5653)

In [19]:
clusters.labels_[:5]

array([27, 61, 82, 93, -1])

In [20]:
articles_df['clusterLabel'] = clusters.labels_
articles_df.head()

,Unnamed: 0,source,author,title,description,url,urlToImage,publishedAt,content,clusterLabel
0,0,"{'id': None, 'name': 'The Guardian'}",Dan Milmo Global technology editor,Is the bird really freed? A look back at six m...,As we approach half a year of Musk’s acquisiti...,https://www.theguardian.com/technology/2023/ap...,https://i.guim.co.uk/img/media/cac584cb9c538ef...,2023-04-16T05:00:03Z,On 28 October 2022 Elon Musk tweeted the bird ...,27
1,1,"{'id': None, 'name': 'Slashdot.org'}",EditorDavid,Remote Working Increases VC Investments in Oth...,Silicon Valley had $74.9 billion in venture-ca...,https://it.slashdot.org/story/23/04/15/1730234...,https://a.fsdn.com/sd/topics/business_64.png,2023-04-16T11:34:00Z,Silicon Valley had $74.9 billion in venture-ca...,61
2,2,"{'id': None, 'name': 'XDA Developers'}",Adam Conway,Despite the best efforts of Google Play Protec...,The Google Play Store is the one-stop shop for...,https://www.xda-developers.com/google-play-sto...,https://static1.xdaimages.com/wordpress/wp-con...,2023-04-16T15:00:17Z,"If you own an Android smartphone, then the pla...",82
3,3,"{'id': None, 'name': 'MakeUseOf'}",Adaeze Uche,5 Myths About Crypto Regulations Debunked,"There are a lot of news, stories, and myths ar...",https://www.makeuseof.com/crypto-regulations-m...,https://static1.makeuseofimages.com/wordpress/...,2023-04-16T13:30:17Z,Crypto assets have existed for over a decade b...,93
4,4,"{'id': None, 'name': 'Biztoc.com'}",newsbtc.com,"OKB Unfazed By BTC's Retreat, Gains 50% In Wee...","OKB, the utility token of popular cryptocurren...",https://biztoc.com/x/5032dbc37cfd67c3,https://c.biztoc.com/p/5032dbc37cfd67c3/og.webp,2023-04-16T10:12:06Z,"OKB, the utility token of popular cryptocurren...",-1


In [23]:
cluster_27 = articles_df['title'][articles_df['clusterLabel'] == 27][:10].values.tolist()
cluster_27

['Is the bird really freed? A look back at six months of Musk’s Twitter reign',
 '234 Million Shiba Inu (SHIB) Tokens Burned as Network Maintains High Load',
 'Shiba Inu Spikes On 60% Spurt In Volume',
 'Litecoin Network Reaches ATH in Hashrate, Boosting Security and Stability',
 'Is the bird really freed? A look back at six months of Musk’s Twitter reign',
 'Koyo Token Holders Surpasses 1,170 Amid Increased Shiba Inu Burns',
 '234 Million Shiba Inu (SHIB) Tokens Burned as Network Maintains High Load',
 'Lead Shiba Inu Developer Stresses More $SHIB Burns Are Needed to ‘Move Price’',
 'Shibarium’s Puppynet Testnet: Massive Interest with 5.8M Wallets and 3.2M+ Transactions',
 "Dogecoin Creator Finally 'Reveals' How DOGE Is Related to Bitcoin"]